<a href="https://colab.research.google.com/github/AmineBAA/Deep-Insights_Neural-Network/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

## 1. Import Libraries

In [ ]:
import torchvision
from torchvision import models
from torchvision import transforms
import torch.optim as optim
import torch
import torch.nn as nn
from torchvision.datasets import ImageNet
from torch.utils.data import DataLoader
import torch.nn.functional as F


## 2. Import dataset

In [ ]:
# Load and preprocess the CIFAR-10 dataset
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)


100%|██████████| 170498071/170498071 [00:02<00:00, 68020247.89it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## 3. Neural Network Contruction

In [ ]:

# Define a CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
# Define the loss function and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [ ]:
# Train the network
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.196
[1,  4000] loss: 1.862
[1,  6000] loss: 1.682
[1,  8000] loss: 1.571
[1, 10000] loss: 1.529
[1, 12000] loss: 1.488
[2,  2000] loss: 1.411
[2,  4000] loss: 1.391
[2,  6000] loss: 1.372
[2,  8000] loss: 1.343
[2, 10000] loss: 1.312
[2, 12000] loss: 1.292
[3,  2000] loss: 1.212
[3,  4000] loss: 1.208
[3,  6000] loss: 1.202
[3,  8000] loss: 1.213
[3, 10000] loss: 1.205
[3, 12000] loss: 1.184
[4,  2000] loss: 1.100
[4,  4000] loss: 1.096
[4,  6000] loss: 1.139
[4,  8000] loss: 1.110
[4, 10000] loss: 1.113
[4, 12000] loss: 1.112
[5,  2000] loss: 1.027
[5,  4000] loss: 1.040
[5,  6000] loss: 1.037
[5,  8000] loss: 1.046
[5, 10000] loss: 1.056
[5, 12000] loss: 1.048
[6,  2000] loss: 0.956
[6,  4000] loss: 0.983
[6,  6000] loss: 0.987
[6,  8000] loss: 0.984
[6, 10000] loss: 0.987
[6, 12000] loss: 1.013
[7,  2000] loss: 0.908
[7,  4000] loss: 0.918
[7,  6000] loss: 0.923
[7,  8000] loss: 0.975
[7, 10000] loss: 0.937
[7, 12000] loss: 0.956
[8,  2000] loss: 0.873
[8,  4000] 

## 4. Testing

In [ ]:
# Test the network on the test data
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 61 %
